# Building simulation for NET 01
This notebook summarizes the simulation construction and running for net 01

In [ ]:
# Defining network file
NET_FILE = ".\data/grid01-bcn_06.net.xml"

### Demand generation
Endogenous demand based on ACTIVITYGEN

In [ ]:
## Config file for parsing demographic data:

ACTIVITYGEN_STAT_FILE = ".\data/activitygen-bcn_06.stat.xml"

## Basic trip output file (without routing) as a resukt of the demand generation based on ACTIVITYGEN.
TRIPS_FROM_ACTIVITYGEN_STAT = ".\data/activitygen-bcn_06.trips.rou.xml"

## Routes output files from ACTIVITYGEN trips
ROUTES_FROM_DUAROUTER_FROM_ACTIVITYGEN = ".\data/activitygen-bcn_06.rou.xml"

In [ ]:
# Calling ACTIVITYGEN tool in SUMO
# Random seed set for reproductibility purposes

!activitygen --net-file {NET_FILE} \
            --stat-file {ACTIVITYGEN_STAT_FILE} \
            --output-file {TRIPS_FROM_ACTIVITYGEN_STAT} \
            --seed 25101987

In [ ]:
# Routing trips from ACTIVITYGEN
# CH routing algorithm is used

!duarouter --net-file {NET_FILE} \
            --route-files {TRIPS_FROM_ACTIVITYGEN_STAT} \
            --output-file {ROUTES_FROM_DUAROUTER_FROM_ACTIVITYGEN} \
            --routing-algorithm CH \
            --ignore-errors

Passing through traffic from random trips between fringe edges

In [ ]:
# creation of hourly distribution of trips based on ACTIVITYGEN output hourly histogram
dist = [209, 226, 263, 472, 1028, 2146, 3679, 5567, 6573, 6539, 5652, 4762, 4409, 4733, 5041, 5450, 6065, 6499, 6266, 5337, 3953, 2605, 1660, 1164]
total_random_trips = 280000
factor = total_random_trips/sum(dist)
random_dist = [int(element * factor) for element in dist]

In [ ]:
# Creating iteratively random trips accordingly to the hourly demand
## CREATE RANDOM FOLDER if it does not exist
import os
if not os.path.exists('data/random'):
    os.makedirs('data/random')

for i in range(len(random_dist)):
    begin = 3600*i
    end = 3600*(i+1)
    trips = random_dist[i]
    period = (end-begin)/trips
    trip_prefix = str(str(i) + "h")
    output_folder_1 = "data/random/random_h_"
    output_folder_2_trips = "-bcn_06.trips.rou.xml"
    output_folder_2_routes = "-bcn_06.rou.xml"

    BEGIN_ARG = str("-b " + str(begin))
    END_ARG = str("-e " + str(end))
    PERIOD_ARG = str("-p" + str(period))
    OUT_TRIPS_ARG = str("-o" + output_folder_1 + str(i) + output_folder_2_trips)
    OUT_ROUTES_ARG = str("-r" + output_folder_1 + str(i) + output_folder_2_routes)
    
    # bash SUMO command for RANDOM trips
    # Adjust the installation path for SUMO
    !python "C:\Program Files (x86)\Eclipse\Sumo\tools\randomTrips.py" -n {NET_FILE} \
    --prefix={trip_prefix} \
    --fringe-factor 100 \
    {BEGIN_ARG} \
    {END_ARG} \
    {PERIOD_ARG} \
    --binomial 10 \
    "{OUT_TRIPS_ARG}" \
    --edge-permission=passenger

In [ ]:
# Merging all random routing files i|n a single file
import os
list_trips_files = []
for file in os.listdir(r"data/random/"):
    if file.endswith(".trips.rou.xml"):
        list_trips_files.append(str("data/random/" + file))
list_trips_files_str = ' '.join([str(str(elem) + ',') for elem in list_trips_files])
list_trips_files_str = list_trips_files_str.replace(" ","")

SINGLE_RANDOM_ROUTES_FILE = "data/random/random_ALL.rou.xml"

!duarouter --net-file {NET_FILE} \
--route-files {list_trips_files_str[0:-1]} \
--output-file {SINGLE_RANDOM_ROUTES_FILE} \
--routing-algorithm CH \
--ignore-errors

### Simulation configuration file
A XML configuration file () is set for taking all the demand inputs, the net, and specifying the output files for results colletions.